In [4]:
#using afids/afids-auto/afids-auto-train/workflow/scripts/reg_qc.py script

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import base64
import os
import re
from glob import glob
from io import BytesIO, StringIO
from pathlib import Path
from tempfile import TemporaryDirectory
from uuid import uuid4

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from nilearn import plotting
from nilearn.datasets import load_mni152_template
from svgutils.compose import Unit
from svgutils.transform import GroupElement, SVGFigure, fromstring



In [6]:

def svg2str(display_object, dpi):
    """Serialize a nilearn display object to string."""

    image_buf = StringIO()
    display_object.frame_axes.figure.savefig(
        image_buf, dpi=dpi, format="svg", facecolor="k", edgecolor="k"
    )
    return image_buf.getvalue()


def extract_svg(display_object, dpi=250):
    """Remove the preamble of the svg files generated with nilearn."""
    image_svg = svg2str(display_object, dpi)

    image_svg = re.sub(' height="[0-9]+[a-z]*"', "", image_svg, count=1)
    image_svg = re.sub(' width="[0-9]+[a-z]*"', "", image_svg, count=1)
    image_svg = re.sub(
        " viewBox", ' preseveAspectRation="xMidYMid meet" viewBox', image_svg, count=1
    )
    start_tag = "<svg "
    start_idx = image_svg.find(start_tag)
    end_tag = "</svg>"
    end_idx = image_svg.rfind(end_tag)

    # rfind gives the start index of the substr. We want this substr
    # included in our return value so we add its length to the index.
    end_idx += len(end_tag)

    return image_svg[start_idx:end_idx]


def clean_svg(fg_svgs, bg_svgs, ref=0):
    # Find and replace the figure_1 id.
    svgs = bg_svgs + fg_svgs
    roots = [f.getroot() for f in svgs]

    sizes = []
    for f in svgs:
        viewbox = [float(v) for v in f.root.get("viewBox").split(" ")]
        width = int(viewbox[2])
        height = int(viewbox[3])
        sizes.append((width, height))
    nsvgs = len([bg_svgs])

    sizes = np.array(sizes)

    # Calculate the scale to fit all widths
    width = sizes[ref, 0]
    scales = width / sizes[:, 0]
    heights = sizes[:, 1] * scales

    # Compose the views panel: total size is the width of
    # any element (used the first here) and the sum of heights
    fig = SVGFigure(Unit(f"{width}px"), Unit(f"{heights[:nsvgs].sum()}px"))

    yoffset = 0
    for i, r in enumerate(roots):
        r.moveto(0, yoffset, scale_x=scales[i])
        if i == (nsvgs - 1):
            yoffset = 0
        else:
            yoffset += heights[i]

    # Group background and foreground panels in two groups
    if fg_svgs:
        newroots = [
            GroupElement(roots[:nsvgs], {"class": "background-svg"}),
            GroupElement(roots[nsvgs:], {"class": "foreground-svg"}),
        ]
    else:
        newroots = roots

    fig.append(newroots)
    fig.root.attrib.pop("width", None)
    fig.root.attrib.pop("height", None)
    fig.root.set("preserveAspectRatio", "xMidYMid meet")

    with TemporaryDirectory() as tmpdirname:
        out_file = Path(tmpdirname) / "tmp.svg"
        fig.save(str(out_file))
        # Post processing
        svg = out_file.read_text().splitlines()

    # Remove <?xml... line
    if svg[0].startswith("<?xml"):
        svg = svg[1:]

    # Add styles for the flicker animation
    if fg_svgs:
        svg.insert(
            2,
            """\
<style type="text/css">
@keyframes flickerAnimation%s { 0%% {opacity: 1;} 100%% { opacity:0; }}
.foreground-svg { animation: 1s ease-in-out 0s alternate none infinite running flickerAnimation%s;}
.foreground-svg:hover { animation-play-state: running;}
</style>"""
            % tuple([uuid4()] * 2),
        )

    return svg


def sorted_nicely(data, reverse=False):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split("([0-9]+)", key)]

    return sorted(data, key=alphanum_key, reverse=reverse)



In [20]:


    
def output_html(gad_img_list, nongad_rigid_img, nongad_affine_img, output_html):
    html_list = []
    for index in range(len(gad_img_list)):#loop over number of files in gad list
        isub = os.path.basename(gad_img_list[index]).split("_")[0]
        ##################################displaying nongad rigid image as foreground #######################
        nongad_rigid = nib.load(nongad_rigid_img[index])
        nongad_rigid = nib.Nifti1Image(
            nongad_rigid.get_fdata().astype(np.float32),
            header= nongad_rigid.header,
            affine=nongad_rigid.affine,
        )
        plot_args_ref = {"dim": 0} #dim adjustss the brifhtness, with -2 being max brightness and 2 being max dimness
        display = plotting.plot_anat( #class that can extract vector graphics from image: plotting gad image
            nongad_rigid, #nongad rigid image 
            display_mode="ortho",
            draw_cross=False,
            cut_coords=[0, 0, 40], #taking slice close to centre, coronal, sagittal and frontal
            **plot_args_ref, # ** upacks the dict
        )
        fg_svgs = [fromstring(extract_svg(display, 300))] #rescaling for nongad rigid 
        display.close()

        #displaying gad image as background 
        gad_img = nib.load(gad_img_list[index]) 
        
        gad_img= nib.Nifti1Image(
            gad_img.get_fdata().astype(np.float32),
            header=gad_img.header,
            affine=gad_img.affine,
        )
        display = plotting.plot_anat(
            gad_img, #gad image
            display_mode="ortho",
            draw_cross=False,
            cut_coords=[0, 0, 40],
            **plot_args_ref,
        )
        bg_svgs = [fromstring(extract_svg(display, 300))]#rescaling for gad (background)
        display.close()

       
        final_svg_rigid= "\n".join(clean_svg(fg_svgs, bg_svgs))#plotting them overtop of each other, and brings them in and out

        anat_params = {
            "vmin": nongad_rigid.get_fdata(dtype="float32").min(),
            "vmax": nongad_rigid.get_fdata(dtype="float32").max(),
            "cmap": plt.cm.gray,
            "interpolation": "none",
            "draw_cross": False,
        }
        #to plot contours of gad image on top of nongad rigidly transformed scan
        display = plotting.plot_anat(nongad_rigid, **anat_params)
        display.add_contours(gad_img, colors="r", alpha=0.7, linewidths=0.8)
        
        tmpfile = BytesIO()
        display.savefig(tmpfile, dpi=300)
        display.close()
        tmpfile.seek(0)
        encoded_rigid = base64.b64encode(tmpfile.getvalue())

        ##################################displaying nongad affine image as foreground #######################
        nongad_affine = nib.load(nongad_affine_img[index])
        nongad_affine = nib.Nifti1Image(
            nongad_affine.get_fdata().astype(np.float32),
            header= nongad_affine.header,
            affine=nongad_affine.affine,
        )
        plot_args_ref = {"dim": 0} #specify dicdctionary , dim can adjust the brifhtness can go up to 2
        display = plotting.plot_anat( #class that can extract vector graphics from image: plotting gad image
            nongad_affine, #nongad rigid image 
            display_mode="ortho",
            draw_cross=False,
            cut_coords=[0, 0, 40], #taking slice close to centre
            **plot_args_ref, # ** upacks the dict
        )
        fg_svgs = [fromstring(extract_svg(display, 300))] #rescaling for nongad affine
        display.close()

        #displaying gad image as background 
        gad_img = nib.load(gad_img_list[index])
        gad_img= nib.Nifti1Image(
            gad_img.get_fdata().astype(np.float32),
            header=gad_img.header,
            affine=gad_img.affine,
        )
        display = plotting.plot_anat(
            gad_img, #gad image
            display_mode="ortho",
            draw_cross=False,
            cut_coords=[0, 0, 40],
            **plot_args_ref,
        )
        bg_svgs = [fromstring(extract_svg(display, 300))]#rescaling for gad (background)
        display.close()

        
        final_svg_affine= "\n".join(clean_svg(fg_svgs, bg_svgs))#plotting them overtop of each other, and brings them in and out


        anat_params = {
            "vmin": nongad_affine.get_fdata(dtype="float32").min(),
            "vmax": nongad_affine.get_fdata(dtype="float32").max(),
            "cmap": plt.cm.gray,
            "interpolation": "none",
            "draw_cross": False,
        }
        #to plot contours of gad image on top of nongad affine transformed scan
        display = plotting.plot_anat(nongad_affine, **anat_params)
        display.add_contours(gad_img, colors="r", alpha=0.7, linewidths=0.8)
        
        tmpfile = BytesIO()
        display.savefig(tmpfile, dpi=300)
        display.close()
        tmpfile.seek(0)
        encoded_affine = base64.b64encode(tmpfile.getvalue())

        html_list.append(f"""
                <center>
                    <h1 style="font-size:42px">{isub}</h1>
                    <h3 style="font-size:42px">Rigid transformation: Nongad to Gad space</h3>
                    <p>{final_svg_rigid}</p>
                    <p><img src="data:image/png;base64, {encoded_rigid.decode("utf-8")}" width=1600 height=600></p>
                    <h3 style="font-size:42px">Affine transformation: Nongad to Gad space</h3>
                    <p>{final_svg_affine}</p>
                    <hr style="height:4px;border-width:0;color:black;background-color:black;margin:30px;">
                    <p><img src="data:image/png;base64, {encoded_affine.decode("utf-8")}" width=1600 height=600></p>
                </center>"""
        )
        print(f"Done {isub}")

    html_string = "".join(html_list)
    message = f"""<html>
            <head></head>
            <body>{html_string}</body>
            </html>"""

    with open(output_html, "w") as fid:
        fid.write(message)

if __name__ == "__main__":
    input_gad_dirs= sorted(glob(f"/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/greedy/*/*_acq-gad_resampled_T1w.nii.gz")) #list of gad images paths
    input_nongad_rigid= glob(f"/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/greedy/*/*rigid_resliced_T1w.nii.gz") #list of rigid transformed nongad image paths
    input_nongad_affine=  glob(f"/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/greedy/*/*affine_resliced_T1w.nii.gz") #list of affine transformed nongad image paths
    output_html_file= "/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/registration_QC.html"
    output_html(input_gad_dirs,input_nongad_rigid,input_nongad_affine,output_html_file)

   

Done sub-P001
Done sub-P002
Done sub-P003


4
